In [ ]:
#qupath edge detection results didn't format columns in for simple read into dataframe
#so reading lines using tab separator into df
#module using glob to iterate through image files

#backend to be buried away:

#read cell detection data
#return centroid nucleus location as  x,y pixel coordinates
#calculate channel labels as ratios and magnitude relatvie to max. channel value
#tidy column labels

In [1]:
import pandas as pd
from tqdm import tqdm
import glob

**MODULES TO LOOP THROUGH 5 LABEL IMAGE FILES**

In [4]:
#5 label column headers
df_columns = ['ï»¿Image', 'Name', 'Class', 'Parent', 'ROI', 'Centroid X Âµm',
       'Centroid Y Âµm', 'Nucleus: Area', 'Nucleus: Perimeter',
       'Nucleus: Circularity', 'Nucleus: Max caliper', 'Nucleus: Min caliper',
       'Nucleus: Eccentricity', 'Nucleus: DRAQ5 Lambda mean',
       'Nucleus: DRAQ5 Lambda sum', 'Nucleus: DRAQ5 Lambda std dev',
       'Nucleus: DRAQ5 Lambda max', 'Nucleus: DRAQ5 Lambda min',
       'Nucleus: DRAQ5 Lambda range', 'Nucleus: EBFP2 Lambda mean',
       'Nucleus: EBFP2 Lambda sum', 'Nucleus: EBFP2 Lambda std dev',
       'Nucleus: EBFP2 Lambda max', 'Nucleus: EBFP2 Lambda min',
       'Nucleus: EBFP2 Lambda range', 'Nucleus: T-sapphire Lambda mean',
       'Nucleus: T-sapphire Lambda sum', 'Nucleus: T-sapphire Lambda std dev',
       'Nucleus: T-sapphire Lambda max', 'Nucleus: T-sapphire Lambda min',
       'Nucleus: T-sapphire Lambda range', 'Nucleus: Venus Lambda mean',
       'Nucleus: Venus Lambda sum', 'Nucleus: Venus Lambda std dev',
       'Nucleus: Venus Lambda max', 'Nucleus: Venus Lambda min',
       'Nucleus: Venus Lambda range', 'Nucleus: mOrange2 Lambda mean',
       'Nucleus: mOrange2 Lambda sum', 'Nucleus: mOrange2 Lambda std dev',
       'Nucleus: mOrange2 Lambda max', 'Nucleus: mOrange2 Lambda min',
       'Nucleus: mOrange2 Lambda range', 'Nucleus: Katushka Lambda mean',
       'Nucleus: Katushka Lambda sum', 'Nucleus: Katushka Lambda std dev',
       'Nucleus: Katushka Lambda max', 'Nucleus: Katushka Lambda min',
       'Nucleus: Katushka Lambda range']

df = pd.DataFrame(columns=df_columns)

In [5]:
#use glob to iter through image folder
#concat rows from all images into single df

for path in tqdm(glob.glob(r"C:\Users\mrq2226\Desktop\imaging\5_dim_images\*")):
    file = open(path,'r')
    all_lines = file.readlines()

    for line in all_lines[1:]:
        split_line = line.split('\t')
        split_line[-1] = split_line[-1][:-1]
        line_dict = dict(zip(df_columns,split_line))
        df = df.append(line_dict, ignore_index=True)
    
        file.close()

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [01:45<00:00, 13.15s/it]


**READ SINGLE 5 LABEL IMAGE**

In [2]:
file = open(r'5_dim_images\Image_009_measurements.txt','r')
all_lines = file.readlines()

df_columns = all_lines[0].split('\t')
df_columns[-1] = df_columns[-1][:-1]

df = pd.DataFrame(columns = df_columns)

for line in tqdm(all_lines[1:]):
    split_line = line.split('\t')
    split_line[-1] = split_line[-1][:-1]
    line_dict = dict(zip(df_columns,split_line))
    df = df.append(line_dict, ignore_index=True)
    
file.close()

#return values as floats

df = df[df.columns[5:]].copy()
for column in df.columns:
    df[column] = df[column].apply(lambda x : float(x))
    
df = df.fillna(0)

100%|██████████████████████████████████████████████████████████████████████████████| 1632/1632 [00:19<00:00, 84.60it/s]


**PREPROCESS**

In [3]:
#want to define centroid_x,y as pixel rather than distance
#need image size and (pixel size or #pixels 

df['centroid_x'] = df['Centroid X Âµm']#*image_size/image_dim
df['centroid_y'] = df['Centroid Y Âµm']#*image_size/image_dim

df['centroid_x'] = df['centroid_x'].apply(lambda x:int(x))
df['centroid_y'] = df['centroid_y'].apply(lambda x:int(x))

In [4]:
#COPY df of 5 channel image
df = df[['centroid_x',
        'centroid_y',
        'Nucleus: DRAQ5 Lambda mean',#nucleus
        'Nucleus: EBFP2 Lambda mean',#blue
        'Nucleus: Venus Lambda mean',#green
        'Nucleus: T-sapphire Lambda mean',#cyan
        'Nucleus: mOrange2 Lambda mean',#orange
        'Nucleus: Katushka Lambda mean'#red
        ]].copy()

df = df.rename(columns = {'Nucleus: DRAQ5 Lambda mean':'channel_nucleus',
                         'Nucleus: Venus Lambda mean':'channel_green',
                         'Nucleus: EBFP2 Lambda mean':'channel_blue',
                         'Nucleus: Katushka Lambda mean':'channel_red',
                         'Nucleus: mOrange2 Lambda mean':'channel_orange',
                         'Nucleus: T-sapphire Lambda mean':'channel_cyan'})

In [5]:
#floor values in lowest n_tile to zero for each channel
#could define normal distribution of channel values and floor < 2 SD?

for column in ['channel_blue', 'channel_green', 'channel_cyan', 'channel_orange','channel_red']:
    
    n_tile = 20
    
    def floorValues(x):
        max_value = df[column].max()
        if x < max_value/n_tile:
            return 0
        else:
            return x
        
    df[column] = df[column].apply(lambda x:floorValues(x))

In [6]:
#calcuate channels as ratio of total cell magnitidue
df['sum'] = df['channel_blue']+df['channel_green']+df['channel_cyan']+df['channel_orange']+df['channel_red']

df['ratio_red'] = df['channel_red']/df['sum']
df['ratio_green'] = df['channel_green']/df['sum']
df['ratio_blue'] = df['channel_blue']/df['sum']
df['ratio_cyan'] = df['channel_cyan']/df['sum']
df['ratio_orange'] = df['channel_orange']/df['sum']

#drop all rows where sum is equal to zero
df = df[df['sum'] != 0].reset_index().drop(columns='index')

In [7]:
#calcuate channels as fraction of max channel magnitude
#may be useful for analysing not only ratio of channel labels but magnitude of channel value
df['magnitude_red'] = df['channel_red']/df['channel_red'].max()
df['magnitude_green'] = df['channel_green']/df['channel_green'].max()
df['magnitude_blue'] = df['channel_blue']/df['channel_blue'].max()
df['magnitude_cyan'] = df['channel_cyan']/df['channel_cyan'].max()
df['magnitude_orange'] = df['channel_orange']/df['channel_orange'].max()

In [8]:
#add cell_index for mapping
df = df.reset_index()
df.insert(0, 'cell_index', df['index'].apply(lambda x:'cell_'+str(x)))
df = df.drop(columns='index')

In [9]:
df.to_csv('test files/qupath_measures.csv',index=False)

**3 LABEL IMAGE MODULES**

In [5]:
#qupath results for three labels
#drop extra columns and rename channel columns

df = df[['Centroid X pixel',
        'Centroid Y pixel',
        'Nucleus: Ch3-T1 mean',#nucleus
        'Nucleus: Ch2 GaAsP-T3 mean',#red
        'Nucleus: Ch3-T2 mean',#green
        'Nucleus: Ch1-T4 mean',#blue
        'Nucleus: Area',
        'Nucleus: Circularity']].copy()

df = df.rename(columns = {'Nucleus: Ch3-T1 mean':'channel_nucleus',
                            'Nucleus: Ch3-T2 mean':'channel_green',
                            'Nucleus: Ch1-T4 mean':'channel_blue',
                            'Nucleus: Ch2 GaAsP-T3 mean':'channel_red'})

In [6]:
#add sum of channels column
df['channel_sum'] = df['channel_red']+df['channel_green']+df['channel_blue']

In [7]:
#add columns of channel value as fraction of total
#all cells to be calculated as ratios

df['ratio_red'] = (df['channel_red']/df['channel_sum'])
df['ratio_green'] = (df['channel_green']/df['channel_sum'])
df['ratio_blue'] = (df['channel_blue']/df['channel_sum'])

In [14]:
#add channel RGB image value as ratio of sum of all channels
#e.g. R=1,G=0,B=0 ratio would be rgb red = 255,rgb green = 0,rgb blue = 0
#and  R=1,G=1,B=1 ratio would be rgb red = 85,rgb green = 85,rgb blue = 85

df['rgb_red'] = df['ratio_red']*255
df['rgb_green'] = df['ratio_green']*255
df['rgb_blue'] = df['ratio_blue']*255

In [15]:
#calculate channel value as relative to channel max value
df['rel_red'] = df['channel_red']/df['channel_red'].max()
df['rel_green'] = df['channel_green']/df['channel_green'].max()
df['rel_blue'] = df['channel_blue']/df['channel_blue'].max()

In [16]:
df = df.rename(columns={'Centroid X pixel':'centroid_x','Centroid Y pixel':'centroid_y','Nucleus: Area':'nucleus_area','Nucleus: Circularity':'nucleus_circularity'})

In [17]:
df = df[['centroid_x','centroid_y','channel_nucleus','nucleus_area','nucleus_circularity',
              'channel_red','channel_green','channel_blue','channel_sum','ratio_red','ratio_green',
              'ratio_blue','rgb_red','rgb_green','rgb_blue','rel_red','rel_green','rel_blue']].copy()

In [18]:
#add cell_index
df = df.reset_index()
df.insert(0, 'cell_index', df['index'].apply(lambda x:'cell_'+str(x)))
df = df.drop(columns='index')

In [20]:
df.to_csv('test files/detected_results.csv',index=False)